In [4]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler,StandardScaler, MinMaxScaler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col,when,regexp_replace,rand
from pyspark.sql.types import IntegerType
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.classification import GBTClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.regression import RandomForestRegressor

import time
import datetime
import numpy as np
import pandas as pd

# Create a SparkSession
spark = SparkSession.builder \
    .appName("RandomForesterSpark") \
    .getOrCreate()

df = spark.read.option("header", "true").option("index", False).csv("D:/project/homecredit/final/v3/final_all_onehotencoding_v3.csv")

df = df.drop('_c0','SK_ID_CURR')

all_columns = df.columns


columns_to_convert = all_columns

for column in columns_to_convert:
    df = df.withColumn(column, col(column).cast(IntegerType()))

df= df.fillna(0)
df= df.drop('_c0','SK_ID_CURR')

In [7]:
train_df, test_df = df.randomSplit([0.7,0.3])
feature_columns =  df.columns
feature_columns.remove("TARGET")

# Assemble the features into a single "features" column
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
train_df = assembler.transform(train_df)
test_df = assembler.transform(test_df)



In [8]:
rd = RandomForestRegressor(
    maxDepth=5,
    featuresCol="features",
    labelCol="TARGET"
)

# Fit the GBTClassifier model to the training data
model = rd.fit(train_df)

# Make predictions on the validation set
prediction = model.transform(test_df)

# Evaluate the model's performance on the validation set
evaluator = BinaryClassificationEvaluator(
    rawPredictionCol = "prediction",
    labelCol="TARGET",
    metricName="areaUnderROC")

print(f'AUC: {evaluator.evaluate(prediction)}')


spark.stop()


AUC: 0.6756616522465627
